# ETL Silver → Gold
Popula o Star Schema no PostgreSQL a partir dos dados limpos do Silver Layer.

## 1. Imports e configurações

In [1]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fake-real-job",
    "user": "postgres",
    "password": "senha123",
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

print("=" * 60)
print("DIAGNOSTICO PRE-ETL (SILVER_JOBS -> GOLD)")
print("=" * 60)

cur.execute("SELECT COUNT(*) FROM public.silver_jobs;")
total = cur.fetchone()[0]

cur.execute("SELECT COUNT(*) FROM public.silver_jobs WHERE salary_avg IS NOT NULL;")
com_salario = cur.fetchone()[0]

cur.execute("SELECT COUNT(*) FROM public.silver_jobs WHERE is_fake IS TRUE;")
fake = cur.fetchone()[0]

print(f"Total vagas: {total:,}")
print(f"Com salario: {com_salario:,}")
print(f"Vagas fake: {fake:,}")

print("=" * 60)


DIAGNOSTICO PRE-ETL (SILVER_JOBS -> GOLD)
Total vagas: 2,197
Com salario: 2,197
Vagas fake: 1,085


## 3. Carregamento da base de dados

In [2]:
df = pd.read_sql("SELECT * FROM public.silver_jobs", conn)
df.head()


,job_id,job_title,company_name,required_experience_years,is_fake,fraud_reason,posting_timestamp,application_deadline_timestamp,salary_min,salary_max,salary_avg,remote,country,state
0,148,Data Analyst,Company_650,8,False,None,2023-12-29,2024-10-05,40000,60000,50000,False,London,UK
1,463,HR Coordinator,Anônimo,5,True,No salary info,2023-01-28,2024-02-17,40000,60000,50000,False,Toronto,Canada
2,133,Business Analyst,Company_449,1,False,None,2023-07-24,2024-08-02,60000,80000,70000,True,None,None
3,471,Marketing Manager,Company_886,2,False,None,2023-06-08,2024-09-18,60000,80000,70000,False,London,UK
4,296,HR Coordinator,Company_1145,0,False,None,2023-10-26,2024-01-06,80000,100000,90000,False,London,UK


## 3. Limpando Tabelas

In [3]:
cur.execute("""
    TRUNCATE TABLE
      gold.fact_jpt,
      gold.dim_cmp,
      gold.dim_jtl,
      gold.dim_loc,
      gold.dim_frd
    CASCADE;
""")
conn.commit()

print("Tabelas GOLD limpas")


Tabelas GOLD limpas


## 4. Dimensão Company (dim_company)

In [4]:
companies = (
    df[["company_name"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_cmp (cmp_nam) VALUES (%s)",
    [(c.strip(),) for c in companies["company_name"] if str(c).strip()],
    page_size=5000
)
conn.commit()


## 5. Dimensão Job_title (dim_job_title)

In [5]:
titles = (
    df[["job_title"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_jtl (jtl) VALUES (%s)",
    [(t.strip(),) for t in titles["job_title"] if str(t).strip()],
    page_size=5000
)
conn.commit()


## 6. Dimensão Location (dim_location)

In [6]:
locations = df[["country", "state"]].drop_duplicates()

rows = []
for _, r in locations.iterrows():
    if pd.notna(r["country"]) or pd.notna(r["state"]):
        rows.append((r["country"], r["state"]))

execute_batch(
    cur,
    "INSERT INTO gold.dim_loc (cty, ste) VALUES (%s, %s)",
    rows,
    page_size=5000
)
conn.commit()


## 7. Dimensão Fraud Reason (dim_fraud_Reason)

In [7]:
reasons = (
    df[["fraud_reason"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_frd (frd) VALUES (%s)",
    [(r.strip(),) for r in reasons["fraud_reason"] if str(r).strip()],
    page_size=5000
)
conn.commit()


## 8. Mapeamento das chaves surrogate das dimensões
Nesta etapa, as dimensões da camada Gold são carregadas em memória e
transformadas em dicionários de mapeamento, permitindo converter valores
descritivos (empresa, cargo, localização e motivo de fraude) em chaves
surrogate (_sk) que serão utilizadas como chaves estrangeiras na tabela fato.


In [8]:
cur.execute("SELECT cmp_nam, cmp_srk FROM gold.dim_cmp")
company_map = dict(cur.fetchall())

cur.execute("SELECT jtl, jtl_srk FROM gold.dim_jtl")
title_map = dict(cur.fetchall())

cur.execute("SELECT cty, ste, loc_srk FROM gold.dim_loc")
location_map = {(c, s): k for c, s, k in cur.fetchall()}

cur.execute("SELECT frd, frd_srk FROM gold.dim_frd")
fraud_map = dict(cur.fetchall())


## 9. Carga da tabela fato (fact_job_posting)
Nesta etapa, os registros da camada Silver são transformados em linhas da
tabela fato da camada Gold, substituindo atributos textuais por chaves
surrogate (_srk) das dimensões, consolidando métricas e atributos analíticos
em um modelo estrela.


In [9]:
rows = []

for _, r in df.iterrows():
    rows.append((
        int(r.job_id),
        company_map.get(r.company_name),
        title_map.get(r.job_title),
        location_map.get((r.country, r.state)),
        fraud_map.get(r.fraud_reason),
        r.salary_avg,
        r.remote,
        r.is_fake
    ))

execute_batch(
    cur,
    """
    INSERT INTO gold.fact_jpt (
        job_id,
        cmp_srk,
        jtl_srk,
        loc_srk,
        frd_srk,
        avg,
        rmt,
        is_fake
    )
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """,
    rows,
    page_size=5000
)

conn.commit()
print("FACT carregada com sucesso")


FACT carregada com sucesso


## 10. Validação da carga da camada Gold
Esta etapa realiza uma verificação quantitativa das tabelas da camada Gold
após a execução do ETL, garantindo que as dimensões e a tabela fato foram
populadas corretamente antes do consumo analítico.


In [10]:
print("\nGOLD (apos ETL):")
for t in [
    "gold.dim_cmp",
    "gold.dim_jtl",
    "gold.dim_loc",
    "gold.dim_frd",
    "gold.fact_jpt",
]:
    cur.execute(f"SELECT COUNT(*) FROM {t}")
    print(f"  {t}: {cur.fetchone()[0]:,}")


  gold.dim_cmp: 731
  gold.dim_jtl: 10
  gold.dim_loc: 4
  gold.dim_frd: 3
  gold.fact_jpt: 2,197


## 11. Encerramento do processo ETL
Nesta etapa, os recursos de banco de dados são liberados com o fechamento da
conexão e do cursor, indicando a conclusão bem-sucedida do processo ETL e
garantindo boas práticas de uso de recursos.


In [11]:
cur.close()
conn.close()
print("\nETL finalizado com sucesso!")


ETL finalizado com sucesso!
